# Mainframe for experimentation

In [ ]:
# Standard library imports
from rdflib import Graph, Namespace
from pyshacl import validate
import datetime
import yaml
import os
import pandas as pd
from tqdm.auto import tqdm

# Local imports
from src.testing_utils import parse_validation_report, apply_mutations, flush_context_to_csv
from src.pipeline_core import run_main_pipeline
from src.llm_utils import GeminiExhaustedException

## Initialization Phase


In [ ]:
DOCUMENT_NAME = "parental_leave"
PROMPT_VERSION = 'Reflexion'
GEMINI_MODEL = "gemini-2.5-flash"
CSV_FILE = "Master_Results.csv"
NUM_RUNS = 1

# Fetch last run ID from CSV
df = pd.read_csv(CSV_FILE, usecols=[0])
if df.empty:
    last_run_id = 0
else: 
    last_run_id = df.iloc[:, 0].max()

# This function creates a blank slate for a single run
def initialize_run_context(run_id, doc_name, model_name):
    return {
        # --- Metadata ---
        "Run ID": run_id,
        "Timestamp": datetime.datetime.now().isoformat(sep=" ", timespec="seconds"),
        "Document Name": doc_name,
        "Prompts":  PROMPT_VERSION,
        "Model Name": model_name,
        
        # --- Pipeline Artifacts (Placeholders) ---
        "Service Graph Hash": "N/A",
        "SHACL Graph Hash": "N/A",
        "SHACL Valid Syntax": "N/A",
        "SHACL Error Type": "N/A",
        "SHACL Error Message": "N/A",
        "Execution Time": "N/A",
        
        # --- Scenario Specifics (Will be overwritten per scenario) ---
        "Scenario ID": "N/A",
        "Scenario Description": "N/A",
        "Expected Violation Count": "N/A",
        "Actual Violation Count": "N/A",
        "Violated Shapes": "N/A",
        "Violation Messages": "N/A",
        
        # --- Execution Stats ---       
        "Successfully Executed": False,
    }
    

## Experiment loop begins

In [ ]:
with tqdm(total=NUM_RUNS, desc="Initializing...") as pbar:
    for run_id in range(last_run_id+1, last_run_id+NUM_RUNS+1):
        try:
            # Create artifact directory for this run
            artifact_dir = f"Testing Artifacts/RUN_{run_id}_{DOCUMENT_NAME}" 
            if not os.path.exists(artifact_dir):
                os.makedirs(artifact_dir)
            else:
                raise FileExistsError(f"Artifact directory {artifact_dir} already exists. Aborting to prevent overwriting.")    
            
            ctx = initialize_run_context(run_id, DOCUMENT_NAME, GEMINI_MODEL)
            
            # Run the main pipeline, creating articfacts and updating ctx
            ctx = run_main_pipeline(ctx, artifact_dir, pbar, DOCUMENT_NAME, PROMPT_VERSION, GEMINI_MODEL, run_id)
            
            if ctx["SHACL Valid Syntax"]: # Begin scenario testing only if SHACL syntax is valid
                
                # Load the Golden Citizen (Baseline)
                golden_ttl = f"Citizens/{DOCUMENT_NAME} eligible.ttl"
                golden_graph = Graph()
                golden_graph.parse(golden_ttl, format="turtle")
                golden_graph.bind("", Namespace("http://example.org/schema#"))
                
                # Load SHACL Shapes Graph
                shacl_ttl = f"{artifact_dir}/{DOCUMENT_NAME} shacl shapes.ttl"
                shacl_graph = Graph()
                shacl_graph.parse(shacl_ttl, format="turtle")
                shacl_graph.bind("", Namespace("http://example.org/schema#"))

                # Load the Scenarios from YAML
                with open(f"Citizens/{DOCUMENT_NAME} scenarios.yaml", "r") as f:
                    scenarios = yaml.safe_load(f)

                # Iterate and Apply
                for scn in scenarios:
                    ctx["Scenario ID"] = scn['id']
                    ctx["Scenario Description"] = scn['description']
                    ctx["Expected Violation Count"] = scn['expected_violation_count']
                    
                    # Apply mutations (Returns a NEW graph, leaving golden_graph untouched)
                    mutated_graph = apply_mutations(golden_graph, scn['actions'])

                    # Proceed to Validation 
                    conforms, results_graph, results_text = validate(
                        data_graph=mutated_graph,
                        shacl_graph=shacl_graph,    
                        inference='rdfs',
                    )
                    
                    # Parse validation report
                    parse_result = parse_validation_report(conforms, results_graph, results_text, shacl_graph)
                    ctx["Actual Violation Count"] = parse_result["violation_count"]
                    ctx["Violated Shapes"] = parse_result["failed_shapes"]
                    ctx["Violation Messages"] = parse_result["messages"]
                    
                    # If we made it this far, scenario ends
                    ctx["Successfully Executed"] = True
                    flush_context_to_csv(ctx, CSV_FILE) 
            else:
                # SHACL syntax was invalid, log and move on to the next run
                flush_context_to_csv(ctx, CSV_FILE)
                
        except GeminiExhaustedException:
            print(f"🛑 FATAL: Gemini exhausted too many times. Check your API usage limits.")
            break # Exit the entire run loop
                
        except Exception as e: # something unexpected happened
            ctx["Successfully Executed"] = str(e).replace("\n", " ")[:250] # Truncate long error messages
            flush_context_to_csv(ctx, CSV_FILE)
            
        finally:
            # Report End of the run to console
            status_msg = "Everything went well!" if ctx["Successfully Executed"] is True else "Exited with some errors."
            pbar.write(f"Logged Run {ctx['Run ID']} to CSV. {status_msg}") 
            pbar.update(1) # Increment progress bar by 1
            
